In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings 
from langchain.text_splitter import CharacterTextSplitter 
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate
from langchain import OpenAI, ConversationChain

from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
# Load sentence transformer model
embeddings = OpenAIEmbeddings()


In [4]:
import os 
# Load openai api key 
openai_api_key = os.environ.get('OPENAI_API_KEY')

# Load Pinecone API key 
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Load pinecode api env 
pinecone_api_env = os.environ.get('PINECONE_API_ENV')

In [7]:
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        text = ''
        for page_num in range(reader.numPages):
            text += reader.getPage(page_num).extractText()
    return text

def get_all_pdf_files_in_directory(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.pdf')]

In [8]:
import pinecone

# Initialize a Pinecone client

pinecone.init(api_key=pinecone_api_key, api_env=pinecone_api_env)
#help(pinecone.init)

/home/leonce/.local/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
# Pick a name for the new index
new_index_name = 'semantic-text-search'
existing_index_name = 'mergers-and-acqs'

In [ ]:
# Generate vector embeddings
#model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [9]:
data_directory = "data/"
pdf_files = get_all_pdf_files_in_directory(data_directory)

In [10]:
bnr_report_reader = UnstructuredPDFLoader("data/Annual_Report_2021_22_Web_English_Versio.pdf")
bnr_report_reader_data = bnr_report_reader.load()

#embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
#llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
# conversation = ConversationChain(llm=llm, verbose=True)

2023-04-24 19:11:26.664890: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 19:11:27.333381: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
conversation = ConversationChain(llm=llm, verbose=True)

In [18]:
import openai
import PyPDF2

openai.api_key = openai_api_key

with open('data/Annual_Report_2021_22_Web_English_Versio.pdf', 'rb') as f:
    pdf_reader = PyPDF2.PdfReader(f)
    num_pages = len(pdf_reader.pages)
    context = ''
    for i in range(num_pages):
        print(PdfReader)
        #page = pdf_reader.pages(i)
        #text = page.extractText()
        #context += text

#embedding_model = "text-davinci-003"
#embeddings = openai.Embedding(embedding_model)
#embedded_context = embeddings.encode(context)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225


In [ ]:
prompt = "Generate a summary of the PDF document."
response = openai.Completion.create(
    engine=embedding_model,
    prompt=prompt,
    max_tokens=900,
    temperature=0.9,
    n=1,
    stop=None,
    prompt_context=embedded_context
)
summary = response.choices[0].text
print(summary)

In [ ]:
import os
import faiss
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import OpenAI, ConversationChain
import time

# Load OpenAI API key from environment
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
import time

questions = [
  "What are the key challenges that Rwanda faces this year?",
  "Summarize key metrics for Rwanda including the amount of reserves, federal direct investment, inflation, and average food prices"
]

for question in questions:
  start_time = time.time()
  # Generate response using OpenAI
  response = conversation.generate_response(question).text
  # Print response
  print(f"Question: {question}")
  print(f"Response: {response}")
  # Get embeddings for response
  response_emb = embeddings.embed(response)
  # Search index for similar embeddings
  D, I = index.search(response_emb, 5)
  end_time = time.time()
  # Print search results
  print(f"Search results: {I}")
  print(f"Search time: {end_time - start_time} seconds\n")

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(bnr_report_reader_data, OpenAIEmbeddings(openai_api_key=openai_api_key))
docs = faiss_index.similarity_search("What are the top economic challenges facing Rwanda?", k=2)

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

In [ ]:
for doc in docs:
    print(docs[0].page_content)

In [ ]:
# Pick a name for the new index
simple_index_name = 'nationalbank-index'

In [ ]:
# Check whether the index with the same name already exists
if simple_index_name in pinecone.list_indexes():
    pinecone.delete_index(simple_index_name)

In [ ]:
# Create a new index
pinecone.create_index(name=simple_index_name, dimension=128)

In [ ]:
pinecone.list_indexes()

In [ ]:
pinecone.describe_index("pinecone-index")

In [ ]:
def extract_named_entities(text):
    import spacy
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return [(X.text, X.label_) for X in doc.ents]

In [ ]:

# Load SentenceTransformer
from sentence_transformers import SentenceTransformer

In [ ]:
from pprint import pprint
# Define retriever
retriever = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Define index
index = pinecone.Index()

def search_pinecone(query):
    # extract named entities from the query
    ne = extract_named_entities([query])[0]
    # create embeddings for the query
    xq = retriever.encode(query).tolist()
    # query the pinecone index while applying named entity filter
    xc = index.query(xq, top_k=10, include_metadata=True, filter={"named_entities": {"$in": ne}})
    # extract article titles from the search result
    r = [x["metadata"]["title"] for x in xc["matches"]]
    return pprint({"Extracted Named Entities": ne, "Result": r})